# QF620 Project Part III - Static Replication

**Group 8**\
Ee Jing Michelle\
Ishani Maitra\
Jermayne Lim Jie Min\
Lim Fang Yi\
Muhammad Saqif Bin Juhaimee\
Rohen S/O Veera Kumaran

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import matplotlib.pylab as plt
from scipy.integrate import quad
import datetime as dt
from scipy.interpolate import interp1d

## Data Cleaning

In [3]:
rates = pd.read_csv('zero_rates_20201201.csv')
SPX = pd.read_csv('SPX_options.csv')
SPY = pd.read_csv('SPY_options.csv')

SPY['date'] = SPY.date.apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))
SPY['expiry'] = SPY.exdate.apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))

SPX['date'] = SPX.date.apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))
SPX['expiry'] = SPX.exdate.apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))


SPY = SPY[SPY['exdate'] == SPY['exdate'].unique()[1]]
SPX = SPX[SPX['exdate'] == SPX['exdate'].unique()[1]]


SPX['strike_price'] = SPX['strike_price'] / 1000
SPY['strike_price'] = SPY['strike_price'] / 1000


SPX['mid'] = (SPX['best_bid'] + SPX['best_offer'])/2
SPY['mid'] = (SPY['best_bid'] + SPY['best_offer'])/2

SPY.tail()

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,expiry,mid
1061,2020-12-01,20210115,P,535.0,170.18,170.65,A,2021-01-15,170.415
1062,2020-12-01,20210115,P,540.0,175.18,175.65,A,2021-01-15,175.415
1063,2020-12-01,20210115,P,545.0,180.18,180.65,A,2021-01-15,180.415
1064,2020-12-01,20210115,P,550.0,185.18,185.65,A,2021-01-15,185.415
1065,2020-12-01,20210115,P,555.0,190.18,190.65,A,2021-01-15,190.415


In [4]:
# Time to Expiry
today = dt.date(2020, 12, 1)
expiries = [pd.Timestamp(str(x)).date() for x in SPX['exdate'].unique()]
unique_T = [(exdate-today).days/365.0 for exdate in expiries]
print(dict(zip(expiries, unique_T)))

days_to_expiry = (expiries[0] - today).days
print(days_to_expiry)

zero_rate_curve = interp1d(rates['days'], rates['rate'])
rate_45 = zero_rate_curve(days_to_expiry)
print(rate_45)

{datetime.date(2021, 1, 15): 0.1232876712328767}
45
0.20510755555555554


In [5]:
# Parameters
S0_SPY = 366.02
S0_SPX = 3662.45
T = days_to_expiry / 365
r = rate_45/100

#SPX SABR
alpha_SPX = 1.821
beta_SPX = 0.7
rho_SPX = -0.407
nu_SPX = 2.788

#SPY SABR
alpha_SPY = 0.910
beta_SPY = 0.7
rho_SPY = -0.491
nu_SPY = 2.726

# Forward Prices
F_SPY = S0_SPY * np.exp(r * T)  # Forward price
F_SPX = S0_SPX * np.exp(r * T)

In [6]:
#SPY ATM & OTM put and call

#Why do i only take otm and itm options
SPY_call = SPY.copy()[SPY["cp_flag"] == 'C']
SPY_call = SPY_call[SPY_call['strike_price'] >= S0_SPY]

SPY_put = SPY.copy()[SPY["cp_flag"] == 'P']
SPY_put = SPY_put[SPY_put['strike_price'] <= S0_SPY]

new_SPY = pd.concat([SPY_call, SPY_put])

#SPX ATM & OTM put and call
SPX_call = SPX.copy()[SPX["cp_flag"] == 'C']
SPX_call = SPX_call[SPX_call['strike_price'] >= S0_SPX]


SPX_put = SPX.copy()[SPX["cp_flag"] == 'P']
SPX_put = SPX_put[SPX_put['strike_price'] <= S0_SPX]

new_SPX = pd.concat([SPX_call, SPX_put])

## Black-Scholes and Bachelier Model

In [8]:
#Black-Scholes
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

#Implied Call & Put Volatility
def impliedCallVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        BlackScholesCall(S, K, r, x, T),
                        1e-6, 1)
    return impliedVol

def impliedPutVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 1)
    return impliedVol

In [9]:
#Bachelier Model
def BachelierCall(S, K, r, sigma, T):
    x = (np.log( (S - K) / (sigma * np.sqrt(T))))
    return np.exp(-r*T) * ( (S - K)* norm.cdf(x) + ( sigma * np.sqrt(T) ) * norm.pdf(x) )

def BachelierPut(S, K, r, sigma, T):
    x = (np.log( (S - K) / (sigma * np.sqrt(T))))
    return np.exp(-r*T) * ( (K - S)* norm.cdf(-x) + ( sigma * np.sqrt(T) ) * norm.pdf(-x) )

In [10]:
#SABR Model
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

def SABRCall(S0, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S0*np.exp(r*T), K, T, alpha, beta, rho, nu) #sabr implied call
    return BlackScholesCall(S0, K, r, sabr_vol, T)


def SABRPut(S0, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S0*np.exp(r*T), K, T, alpha, beta, rho, nu) #sabr implied vol
    return BlackScholesPut(S0, K, r, sabr_vol, T)


def sabrcallintegrand(K, S0, r, T, alpha, beta, rho, nu):
    price = SABRCall(S0, K, r, alpha, beta, rho, nu, T) / K**2
    return price


def sabrputintegrand(K, S0, r, T, alpha, beta, rho, nu):
    price = SABRPut(S0, K, r, alpha, beta, rho, nu, T) / K**2
    return price

In [11]:
def h(K):
    K = np.where(np.abs(K) < 1e-6, 1e-6, K)
    return K**(1/3) + 1.5 * np.log(K) + 10

def h2(K):
    K = np.where(np.abs(K) < 1e-6, 1e-6, K)
    return -2/9 * K **(-5/3) - 3/2 * K** (-2)

In [12]:
 new_SPY.sort_values(by = 'strike_price' , ascending = False).head()

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,expiry,mid
787,2020-12-01,20210115,C,555.0,0.0,0.01,A,2021-01-15,0.005
786,2020-12-01,20210115,C,550.0,0.0,0.01,A,2021-01-15,0.005
785,2020-12-01,20210115,C,545.0,0.0,0.01,A,2021-01-15,0.005
784,2020-12-01,20210115,C,540.0,0.0,0.01,A,2021-01-15,0.005
783,2020-12-01,20210115,C,535.0,0.0,0.01,A,2021-01-15,0.005


In [13]:
new_SPY['abs_diff'] = np.abs(new_SPY['strike_price'] - S0_SPY) #Finding price closest to forward price
print(new_SPY.nsmallest(5, 'abs_diff'))

new_SPX['abs_diff'] = np.abs(new_SPX['strike_price'] - S0_SPX)
# print(new_SPX.nsmallest(5, 'abs_diff'))

          date    exdate cp_flag  strike_price  best_bid  best_offer  \
997 2020-12-01  20210115       P         366.0     10.06       10.10   
720 2020-12-01  20210115       C         367.0      8.28        8.33   
996 2020-12-01  20210115       P         365.0      9.64        9.68   
721 2020-12-01  20210115       C         368.0      7.74        7.78   
995 2020-12-01  20210115       P         364.0      9.24        9.28   

    exercise_style     expiry     mid  abs_diff  
997              A 2021-01-15  10.080      0.02  
720              A 2021-01-15   8.305      0.98  
996              A 2021-01-15   9.660      1.02  
721              A 2021-01-15   7.760      1.98  
995              A 2021-01-15   9.260      2.02  


In [14]:
# Compute implied volatilities
K_SPX = 3660
K_SPY = 366

SPY_sigma = impliedPutVolatility(S0_SPY , K_SPY , r , 10.08,  T)

SPX_sigma = impliedPutVolatility(S0_SPX , K_SPX , r , 95.15 , T)
print("Sigma for SPY under Black-Scholes Model: " , SPY_sigma)
print("Sigma for SPX under Black-Scholes Model: " , SPX_sigma)

Sigma for SPY under Black-Scholes Model:  0.1977667232964117
Sigma for SPX under Black-Scholes Model:  0.18886028486825987


In [15]:
#Black Scholes Pricing
SPY_BS = np.exp(-r * T) * h(F_SPY) + h2(F_SPY)*(1 / K_SPY**2) * BlackScholesCall(r = r , S = S0_SPY , K = K_SPY, T = T, sigma = SPY_sigma)+ h2(F_SPY)*(1 / K_SPY**2) * BlackScholesPut(r = r , S = S0_SPY , K = K_SPY, T = T, sigma = SPY_sigma)
SPX_BS = np.exp(-r * T) * h(F_SPX) + h2(F_SPX)*(1 / K_SPX**2) * BlackScholesCall(r = r , S = S0_SPX , K = K_SPX, T = T, sigma = SPX_sigma)+ h2(F_SPX)*(1 / K_SPX**2) * BlackScholesPut(r = r , S = S0_SPX , K = K_SPX, T = T, sigma = SPX_sigma)

print(" Pricing of SPY from Black-Scholes Model is :" , SPY_BS)
print(" Pricing of SPX from Black-Scholes Model is :" , SPX_BS)

 Pricing of SPY from Black-Scholes Model is : 26.00165870035046
 Pricing of SPX from Black-Scholes Model is : 37.71527504034801


In [16]:
# Bachelier Pricing
SPY_Bachelier = np.exp(-r * T) * h(F_SPY) + h2(F_SPY)*(1 / K_SPY**2) * BachelierCall(r = r , S = S0_SPY , K = K_SPY, T = T, sigma = SPY_sigma)+ h2(F_SPY)*(1 / K_SPY**2) * BachelierPut(r = r , S = S0_SPY , K = K_SPY, T = T, sigma = SPY_sigma)
SPX_Bachelier = np.exp(-r * T) * h(F_SPX) + h2(F_SPY)*(1 / K_SPX**2) * BachelierCall(r = r , S = S0_SPX , K = K_SPX, T = T, sigma = SPX_sigma)+ h2(F_SPY)*(1 / K_SPX**2) * BachelierPut(r = r , S = S0_SPX , K = K_SPX, T = T, sigma = SPX_sigma)

print(" Pricing of SPY from Bachelier Model is :" , SPY_Bachelier)
print(" Pricing of SPX from Bachelier Model is :" , SPX_Bachelier)

 Pricing of SPY from Bachelier Model is : 26.001658703837276
 Pricing of SPX from Bachelier Model is : 37.715275040349105


In [17]:
#SABR Model
I_SPX_put = quad(lambda x: sabrputintegrand(x, S0_SPX, r, T, alpha_SPX, beta_SPX, rho_SPX, nu_SPX), 1e-6, F_SPX)[0]
I_SPX_call = quad(lambda x: sabrcallintegrand(x , S0_SPX, r, T, alpha_SPX, beta_SPX, rho_SPX, nu_SPX), F_SPX, 5000)[0]

I_SPY_call = quad(lambda x: sabrcallintegrand(x , S0_SPY,r, T, alpha_SPY, beta_SPY, rho_SPY, nu_SPY), F_SPY, 5000)[0]
I_SPY_put = quad(lambda x: sabrputintegrand(x , S0_SPY, r, T, alpha_SPY, beta_SPY, rho_SPY, nu_SPY), 0.0, F_SPY)[0]


SPX_SABR = np.exp(-r * T) * h(F_SPX) + I_SPX_call + I_SPX_put
SPY_SABR = np.exp(-r * T) * h(F_SPY) + I_SPY_call + I_SPY_put

print(" Pricing of SPX from SABR Model is :" , SPX_SABR)
print(" Price SPY from SABR Model is: " , SPY_SABR )

 Pricing of SPX from SABR Model is : 37.71845332801057
 Price SPY from SABR Model is:  26.004673143470693


## Model-Free Variance

In [19]:
SABR_SPX_var = 2*np.exp(r*T)*(I_SPX_put + I_SPX_call)
SABR_SPY_var = 2*np.exp(r*T)*(I_SPY_put + I_SPY_call)

MF_SPX_var = SABR_SPX_var/T
MF_SPY_var = SABR_SPY_var/T

BS_SPY_var = SPY_sigma**2 * T
BS_SPX_var = SPX_sigma**2 * T

BS_SPY_vol = np.sqrt(BS_SPY_var / T )
BS_SPX_vol = np.sqrt(BS_SPX_var / T )

print('Black-Scholes sigma for SPY derived from MF variance is ' , BS_SPY_vol)
print('Black-Scholes sigma for SPX derived from MF variance is ' , BS_SPX_vol)

print('SABR sigma derived from MF variance is: ' , np.sqrt(MF_SPX_var))
print('SABR sigma derived from MF variance is: ' , np.sqrt(MF_SPY_var))

Black-Scholes sigma for SPY derived from MF variance is  0.1977667232964117
Black-Scholes sigma for SPX derived from MF variance is  0.18886028486825987
SABR sigma derived from MF variance is:  0.22709453566060128
SABR sigma derived from MF variance is:  0.22116346179231763
